### To run this model yourself: go to Run -> Run All Cells in the top left menu bar.

In [ ]:
from epxexec.fred_job import fred_job

import pandas as pd
import numpy as np

import networkx as nx

import matplotlib as mpl
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.io as pio
from epxexec.visual.utils import default_plotly_template
pio.templates["epistemix"] = default_plotly_template()
pio.templates.default = "epistemix"

In [ ]:
plt.style.use('seaborn-v0_8-darkgrid')

mpl.rc('axes', grid=True, facecolor="#000533", labelcolor="#F3F3F7")
mpl.rc('figure', facecolor="#000533")
mpl.rc('grid', color="#6C76A8", linewidth=0.7)
mpl.rc("text", color="#F3F3F7")
mpl.rc("xtick", color="#F3F3F7")
mpl.rc("ytick", color="#F3F3F7")

# Idea Evolution and Misinformation

In [ ]:
%%time
job = fred_job("model/main.fred")

In [ ]:
network = job.runs[1].get_csv_output("network_edges.csv")
idea = job.runs[1].get_csv_output('idea_evolution.csv')

## Agent Network

First we'll take a look at a subset of the network generated based on the agent's daily interactions. In the visualization below, each white dot is an agent (network node), and the lines are connections between the agents (network edges). The color and thickness of the lines represents the strength of that connection (network weights). 

In this model, connection strength is randomly drawn from a range of values which varies based on where the connection is made (a higher value range for at-home connections, intermediate range for work or school, lower range for connections made through the community at large). 

In [ ]:
plot_sample = network[network.X!=network.Y].iloc[:500]

G = nx.Graph()

fig1, ax1 = plt.subplots(figsize=(16, 16))
for u,v,x in zip(plot_sample['X'], 
               plot_sample['Y'],plot_sample['weight']):
    G.add_edge(u, v, weight=x)  
    
nx.draw_networkx(G, node_color="#E8ECFC", node_size=80, 
               linewidths=2.5,
               width=(plot_sample.weight+0.2)*8, 
                 edge_color=plt.cm.viridis(plot_sample.weight),
               edgecolors="#000533", with_labels=False, pos=nx.spring_layout(G))

## Idea Spread

This model represents a single "idea" with a pair of (x,y) coordinates. A certain number of randomly selected agents (set by the `n_idea_seeds` variable; default=10) are originators of the idea, beginning with idea coordinates (0,0). They each seek out a friend (drawn from their network links) to share with. Each sharing of the idea slightly garbles the messages (adds noise drawn from `uniform(-0.1,0.1)` to each coordinate).

In the visualization below, we can see just how much the "idea" values evolve with time. Each marker represents a set of values of the idea as heard by an agent, and the markers are color-coded by what simulation day they heard the idea on. ~250 days after the idea appeared, we have agents who report the idea values to be (-0.1, 0.6) and some who report (0.1, -1.9).

In [ ]:
fig,ax = plt.subplots(figsize=(7,6))
tmp_idea = idea[idea.my_origin!=0]
for i in range(len(tmp_idea)):
    node1 = tmp_idea.iloc[i]
    node2 = idea[(idea.ID==node1.my_origin)].iloc[0]
    ax.plot([node1.my_x, node2.my_x], [node1.my_y, node2.my_y], c="w", zorder=0, lw=1)

sc = ax.scatter(idea.my_x, idea.my_y, c=idea.simday, cmap=plt.cm.viridis)
cb = plt.colorbar(sc, label="Simday")

The figure below highlights a single "idea chain" to show just how much a single game of telephone can distort the idea.

In [ ]:
fig,ax = plt.subplots(figsize=(6,6))

idea_line_x = []
idea_line_y = [] 
for j in range(1,10):
    start_id = idea.iloc[j].ID
    for i in range(len(idea)):
        if len(idea[idea.my_origin==start_id])==0:
            break
        node1 = idea[idea.ID==start_id].iloc[0]
        node2 = idea[idea.my_origin==start_id].iloc[0]
        ax.plot([node1.my_x, node2.my_x], 
                [node1.my_y, node2.my_y],
                c="#8C96CA", zorder=0, lw=0.7)

        start_id = node2.ID
        idea_line_x += [node2.my_x]
        idea_line_y += [node2.my_y]

ax.scatter(idea_line_x, idea_line_y, c="#8C96CA", s=2, alpha=0.8)

idea_line_x = []
idea_line_y = [] 
idea_line_c = [] 
start_id = idea.iloc[0].ID
for i in range(len(idea)):
    if len(idea[idea.my_origin==start_id])==0:
        break
    node1 = idea[idea.ID==start_id].iloc[0]
    node2 = idea[idea.my_origin==start_id].iloc[0]
    ax.plot([node1.my_x, node2.my_x], 
            [node1.my_y, node2.my_y],
            c="w", zorder=1, lw=1.2)

    start_id = node2.ID
    idea_line_x += [node2.my_x]
    idea_line_y += [node2.my_y]
    idea_line_c += [node2.simday]
    
ax.scatter(idea_line_x, idea_line_y, c=idea_line_c, cmap=plt.cm.viridis,
           edgecolor="w", s=60, linewidth=1.2)

Finally, we can animate the idea's evolution, starting from the 10 seeds all at (0,0) and watch how the idea changes with each transfer.

In [ ]:
%%capture 

anim_tmp = idea[idea.simday==3].copy()
anim_tmp = anim_tmp.assign(color = ["Old","New","New","New","New","New","New","New","New","New"])


for i in range(4,idea.simday.max()+1):
    tmp1 = idea[idea.simday<i]
    tmp1["simday"] = i
    tmp1 = tmp1.assign(color = "Old")
    
    tmp2 = idea[idea.simday==i]
    tmp2 = tmp2.assign(color = "New")
    
    anim_tmp = pd.concat([anim_tmp, tmp1, tmp2])

In [ ]:
fig = px.scatter(anim_tmp, x="my_x", y="my_y", animation_frame="simday", 
                 animation_group="ID",color="color",
                 range_x=[-1.1,1.1], range_y=[-2.2, 1.6])


fig.update_layout(height=600,)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 100
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 0

fig.update_layout(
    font_family="Epistemix Label",
    title="The Evolution of an Idea",
    title_font_size=24,
)

fig.show()